    

---

# Pandas _(3h30)_


---



# Importation et pré-traitement d'un dataset
---



*Le jeu de données à analyser se trouve à l'adresse suivante : (https://divvy-tripdata.s3.amazonaws.com/index.html). Il y a un fichier par mois, pour un total de 12 fichiers. Chaque fichier contient 13 colonnes avec des types de données variés. Nous allons fusionner les fichiers en un seul et le nommer 'combined_data'.*

In [ ]:
# Importation des bibliothèques
import numpy as np
import pandas as pd
import datetime
import requests
import zipfile
import io
import os
from math import radians, sin, cos, sqrt, atan2

In [ ]:
####### Importation des datasets depuis le serveur AWS dédiéc (A EXECUTER UNE SEULE FOIS !!!!)

# Liste des noms de fichiers
file_names = [
    "202101-divvy-tripdata.zip",
    "202102-divvy-tripdata.zip",
    "202103-divvy-tripdata.zip",
    "202104-divvy-tripdata.zip",
    "202105-divvy-tripdata.zip",
    "202106-divvy-tripdata.zip",
    "202107-divvy-tripdata.zip",
    "202108-divvy-tripdata.zip",
    "202109-divvy-tripdata.zip",
    "202110-divvy-tripdata.zip",
    "202111-divvy-tripdata.zip",
    "202112-divvy-tripdata.zip"
]

# URL de base
base_url = "https://divvy-tripdata.s3.amazonaws.com/"

# Télécharger chaque fichier
for file_name in file_names:

    url = base_url + file_name
    response = requests.get(url)

    # Extraire le fichier Zip dans le dossier local "wild_divvy_data"
    with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
      the_zip.extractall("wild_divvy_data")


In [ ]:
folder_path = 'wild_divvy_data'

# Obtenir une liste de tous les fichiers CSV dans le dossier local
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialiser un DataFrame vide pour stocker les données combinées
combined_data = pd.DataFrame()

# Parcourir chaque fichier CSV et concaténer ses données au DataFrame combiné
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, data], ignore_index=True)

# Afficher les données combinées
combined_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,99103BB87CC6C1BB,electric_bike,2021-08-10 17:15:49,2021-08-10 17:22:44,NaN,NaN,NaN,NaN,41.770000,-87.680000,41.770000,-87.680000,member
1,EAFCCCFB0A3FC5A1,electric_bike,2021-08-10 17:23:14,2021-08-10 17:39:24,NaN,NaN,NaN,NaN,41.770000,-87.680000,41.770000,-87.630000,member
2,9EF4F46C57AD234D,electric_bike,2021-08-21 02:34:23,2021-08-21 02:50:36,NaN,NaN,NaN,NaN,41.950000,-87.650000,41.970000,-87.660000,member
3,5834D3208BFAF1DA,electric_bike,2021-08-21 06:52:55,2021-08-21 07:08:13,NaN,NaN,NaN,NaN,41.970000,-87.670000,41.950000,-87.650000,member
4,CD825CB87ED1D096,electric_bike,2021-08-19 11:55:29,2021-08-19 12:04:11,NaN,NaN,NaN,NaN,41.790000,-87.600000,41.770000,-87.620000,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595058,9397BDD14798A1BA,docked_bike,2021-03-20 14:58:56,2021-03-20 17:22:47,Michigan Ave & Oak St,13042,New St & Illinois St,TA1306000013,41.900960,-87.623777,41.890847,-87.618617,casual
5595059,BBBEB8D51AAD40DA,classic_bike,2021-03-02 11:35:10,2021-03-02 11:43:37,Kingsbury St & Kinzie St,KA1503000043,New St & Illinois St,TA1306000013,41.889177,-87.638506,41.890847,-87.618617,member
5595060,637FF754DA0BD9E1,classic_bike,2021-03-09 11:07:36,2021-03-09 11:49:11,Michigan Ave & Oak St,13042,Clark St & Berwyn Ave,KA1504000146,41.900960,-87.623777,41.977997,-87.668047,member
5595061,F8F43A0B978A7A35,classic_bike,2021-03-01 18:11:57,2021-03-01 18:18:37,Kingsbury St & Kinzie St,KA1503000043,New St & Illinois St,TA1306000013,41.889177,-87.638506,41.890847,-87.618617,member


Pré-traitement
---



1. Afficher les types des colonnes

In [ ]:
combined_data.dtypes

,0
ride_id,object
rideable_type,object
started_at,object
ended_at,object
start_station_name,object
start_station_id,object
end_station_name,object
end_station_id,object
start_lat,float64
start_lng,float64


2. Afficher le nombre de valeurs manquantes par colonne

In [ ]:
combined_data.isna().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,690809
start_station_id,690806
end_station_name,739170
end_station_id,739170
start_lat,0
start_lng,0


3. Identifier et compter les lignes dupliquées pour toutes les colonnes

In [ ]:
combined_data.duplicated().sum()

np.int64(0)

## Traitement

*Commentaire :*

4. Créer un nouveau dataframe en supprimant tous les trajets avec des valeurs manquantes et afficher le nombre de lignes et de colonnes de la nouvelle dataframe

In [ ]:
df = combined_data.dropna(subset=["start_station_name", "start_station_id", "end_station_name", "end_station_id"])
df.shape

(4588302, 13)

## Colonnes supplémentaires et transformation des données

Nous ajouterons la distance parcourue en kilomètres à partir des latitudes et longitudes de départ et d'arrivée données. Nous utiliserons la formule de Haversine.

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
# Rayon de la Terre en kilomètres
    R = 6371.0

    # Convertir la latitude et la longitude de degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculer les différences entre les latitudes et les longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Formule de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculer la distance
    distance = R * c

    return distance


5. Ajouter une nouvelle colonne "distance_travelled_km" en appliquant la fonction haversine sur tous les trajets.

> On rappelle le prototype de la fonction haversine(lat1, lon1, lat2, lon2)



In [ ]:
df['distance_travelled_km'] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].apply(lambda row: haversine(row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]),
    axis=1)
df.head()

/tmp/ipython-input-10-1368779299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_travelled_km'] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].apply(lambda row: haversine(row["start_lat"], row["start_lng"], row["end_lat"], row["end_lng"]),


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km
30,DD06751C6019D865,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535
36,79973DC3B232048F,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306
40,0249AD4B258806AD,classic_bike,2021-08-08 12:59:18,2021-08-08 12:59:55,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,member,0.000000
72,F41EB054E44ACFDA,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000
121,B149E6C71A1C3B14,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000


6. Analyser les statistiques concernant la colonne "distance_travelled_km".

Que remarquez vous?

In [ ]:
df['distance_travelled_km'].describe()

,distance_travelled_km
count,4.588302e+06
mean,2.128845e+00
std,1.879802e+00
min,0.000000e+00
25%,9.038498e-01
50%,1.619725e+00
75%,2.814214e+00
max,3.380018e+01


*Réponse :* Dans le df, il reste des trajets aberrants car d'une durée de 0km.

7. Pour remédier à ce problème, ajouter une nouvelle colonne "ride_duration_s" pour calculer la période du trajet en secondes

In [ ]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['ride_duration_s'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df.head()

/tmp/ipython-input-12-3518405316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['started_at'] = pd.to_datetime(df['started_at'])
/tmp/ipython-input-12-3518405316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ended_at'] = pd.to_datetime(df['ended_at'])
/tmp/ipython-input-12-3518405316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s
30,DD06751C6019D865,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0
36,79973DC3B232048F,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0
40,0249AD4B258806AD,classic_bike,2021-08-08 12:59:18,2021-08-08 12:59:55,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,member,0.000000,37.0
72,F41EB054E44ACFDA,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0
121,B149E6C71A1C3B14,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0


8. Analyser les statistiques concernant la colonne "ride_duration_s".

Que remarquez vous?

In [ ]:
df['ride_duration_s'].describe()

,ride_duration_s
count,4.588302e+06
mean,1.308638e+03
std,1.106550e+04
min,-3.354000e+03
25%,4.170000e+02
50%,7.320000e+02
75%,1.327000e+03
max,3.356649e+06


*Réponse :* Il existe des trajets avec une durée négative.

9. Supprimer les trajets de moins d'une minute

In [ ]:
df1 = df[df['ride_duration_s'] >= 60]
df1.shape

(4528933, 15)

10. Créer une colonne "speed_mph" afin d'analyser la vitesse en calculant la vitesse du km/h vers mph.



In [ ]:
mile = 0.621371
df1['speed_mph'] = (df1['distance_travelled_km'] * mile) / (df1['ride_duration_s'] / 3600)
df1.head()

/tmp/ipython-input-15-186009632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['speed_mph'] = (df1['distance_travelled_km'] * mile) / (df1['ride_duration_s'] / 3600)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph
30,DD06751C6019D865,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,4.407343
36,79973DC3B232048F,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,3.818551
72,F41EB054E44ACFDA,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,0.000000
121,B149E6C71A1C3B14,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,0.000000
123,C41829CD6CC5A8B6,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,0.000000


11. Supprimer les données avec une vitesse supérieure à 45 km/h (~28 mph, vitesse maximale des vélos aux États-Unis).

In [ ]:
df2 = df1[df1['speed_mph']<28]
df2.shape

(4528930, 16)

In [ ]:
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,speed_mph
30,DD06751C6019D865,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,4.407343
36,79973DC3B232048F,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,3.818551
72,F41EB054E44ACFDA,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,0.000000
121,B149E6C71A1C3B14,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,0.000000
123,C41829CD6CC5A8B6,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,0.000000


12. Supprimer les colonnes ("ride_id" et "speed_mph") pour un chargement plus rapide

In [ ]:
df3 = df2.drop(columns = ['ride_id', 'speed_mph'])
df3.shape

(4528930, 14)

13. * Supprimer les espaces de fin pour les colonnes de type chaîne de caractères.

  * Il y a des données qui sont des trajets de test (avec 'test' inclus dans le nom de la station et l'identifiant de la station). Nous supprimerons également ces données.

In [ ]:
# Suppression des espaces de début et fin
col_text = ['rideable_type', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'member_casual']
for colonne in col_text:
    df3[colonne] = df3[colonne].str.strip()

In [ ]:
# Suppression des trajets Tests
col_test = ['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']
df4 = df3.copy()
for colonne in col_test:
    df4 = df4[~df4[colonne].str.contains('test', case=False)]
df4.shape

(4528312, 14)

14. Ajouter les catégories suivantes pour découvrir les motifs basés sur le temps :

* day_of_week : Lundi, Mardi, Mercredi, etc.
* day_type : Jour de semaine ou Week-end
* month : Janvier, Février, Mars, etc.
* season : Hiver, Printemps, Été, Automne

    **Créer une fonction pour day_type**

    **Créer une fonction pour season**

In [ ]:
# Traitement 'day_of_week' et 'month'
df4['day_of_week'] = df4['started_at'].dt.day_name()
df4['month'] = df4['started_at'].dt.month_name()
df4.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,day_of_week,month
30,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,Sunday,August
36,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,Friday,August
72,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,Thursday,August
121,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,Monday,August
123,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,Monday,August


In [ ]:
# Fonction pour day_type
def day_type(jour : str)-> str:
    weekdays=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    if jour in weekdays:
        return "Weekday"
    else:
        return "Weekend"


In [ ]:
#Application de la fonction pour 'day_type'
df4['day_type'] = df4['day_of_week'].apply(day_type)
df4.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,day_of_week,month,day_type
30,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,Sunday,August,Weekend
36,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,Friday,August,Weekday
72,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,Thursday,August,Weekday
121,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,Monday,August,Weekday
123,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,Monday,August,Weekday


In [ ]:
#Fonction pour 'season'
def season(mois : str)-> str:
    winter = ['December', 'January', 'February']
    spring = ['March', 'April', 'May']
    summer = ['June', 'July', 'August']
    if mois in winter:
        return "Winter"
    elif mois in spring:
        return "Spring"
    elif mois in summer:
        return "Summer"
    else :
        return "Fall"

In [ ]:
#Application de la fonction pour 'season'
df4['season'] = df4['month'].apply(season)
df4.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,day_of_week,month,day_type,season
30,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,Sunday,August,Weekend,Summer
36,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,Friday,August,Weekday,Summer
72,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,Thursday,August,Weekday,Summer
121,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,Monday,August,Weekday,Summer
123,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,Monday,August,Weekday,Summer


15. Ajouter la colonne 'route_type' (même 'start_station_name' et 'end_station_name') pour classifier les trajets comme trajet aller-retour ou trajet aller simple.

In [ ]:
#Fonction for 'route_type'
def route_type(station:str)-> str:
    start = station.iloc[0]
    end = station.iloc[1]
    if start == end:
        return "Round trip"
    else:
        return "One-way trip"

In [ ]:
#Application 'route_type'
df4['route_type'] = df4[['start_station_name', 'end_station_name']].apply(route_type, axis=1)
df4.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_travelled_km,ride_duration_s,day_of_week,month,day_type,season,route_type
30,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,Sunday,August,Weekend,Summer,One-way trip
36,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,Friday,August,Weekday,Summer,One-way trip
72,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,Thursday,August,Weekday,Summer,Round trip
121,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,Monday,August,Weekday,Summer,Round trip
123,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,Monday,August,Weekday,Summer,Round trip


16. Ajouter la colonne "ride_duration_min" qui représente la durée du trajet en minutes.

In [ ]:
df4['ride_duration_min'] = df4['ride_duration_s']/60

17. Renommer tes colonnes pour qu'elles soient pretes pour la dataViz

In [ ]:
df4.rename(columns = {
    'rideable_type' : 'bike_type',
    'member_casual' : 'user_type',
    'started_at' : 'start_time',
    'ended_at' : 'end_time',
    'day_of_week' : 'day_of_week',
    'day_type' : 'day_type',
    'month' : 'month',
    'season' : 'season',
    'start_station_name' : 'start_station_name',
    'end_station_name' : 'end_station_name',
    'route_type' : 'route_type',
    'start_lat' : 'start_lat',
    'start_lng' : 'start_lng',
    'end_lat' : 'end_lat',
    'end_lng' : 'end_lng',
    'distance_travelled_km' : 'distance_travelled_km',
    'ride_duration_s' : 'ride_duration_s',
    'ride_duration_min' : 'ride_duration_min'
})

,bike_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,user_type,distance_travelled_km,ride_duration_s,day_of_week,month,day_type,season,route_type,ride_duration_min
30,classic_bike,2021-08-08 17:21:26,2021-08-08 17:25:37,Desplaines St & Kinzie St,TA1306000003,Kingsbury St & Kinzie St,KA1503000043,41.888716,-87.644448,41.889177,-87.638506,member,0.494535,251.0,Sunday,August,Weekend,Summer,One-way trip,4.183333
36,classic_bike,2021-08-27 08:53:52,2021-08-27 09:18:29,Larrabee St & Armitage Ave,TA1309000006,Michigan Ave & Oak St,13042,41.918084,-87.643749,41.900960,-87.623777,member,2.521306,1477.0,Friday,August,Weekday,Summer,One-way trip,24.616667
72,classic_bike,2021-08-12 16:52:09,2021-08-12 16:56:51,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,282.0,Thursday,August,Weekday,Summer,Round trip,4.700000
121,classic_bike,2021-08-23 15:33:04,2021-08-23 16:09:00,Michigan Ave & Oak St,13042,Michigan Ave & Oak St,13042,41.900960,-87.623777,41.900960,-87.623777,casual,0.000000,2156.0,Monday,August,Weekday,Summer,Round trip,35.933333
123,classic_bike,2021-08-23 10:11:09,2021-08-23 10:51:11,Aberdeen St & Jackson Blvd,13157,Aberdeen St & Jackson Blvd,13157,41.877726,-87.654787,41.877726,-87.654787,casual,0.000000,2402.0,Monday,August,Weekday,Summer,Round trip,40.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595057,electric_bike,2021-03-14 01:59:38,2021-03-14 03:13:09,Larrabee St & Armitage Ave,TA1309000006,New St & Illinois St,TA1306000013,41.918129,-87.643749,41.890808,-87.618779,member,3.674160,4411.0,Sunday,March,Weekend,Spring,One-way trip,73.516667
5595058,docked_bike,2021-03-20 14:58:56,2021-03-20 17:22:47,Michigan Ave & Oak St,13042,New St & Illinois St,TA1306000013,41.900960,-87.623777,41.890847,-87.618617,casual,1.202887,8631.0,Saturday,March,Weekend,Spring,One-way trip,143.850000
5595059,classic_bike,2021-03-02 11:35:10,2021-03-02 11:43:37,Kingsbury St & Kinzie St,KA1503000043,New St & Illinois St,TA1306000013,41.889177,-87.638506,41.890847,-87.618617,member,1.656782,507.0,Tuesday,March,Weekday,Spring,One-way trip,8.450000
5595060,classic_bike,2021-03-09 11:07:36,2021-03-09 11:49:11,Michigan Ave & Oak St,13042,Clark St & Berwyn Ave,KA1504000146,41.900960,-87.623777,41.977997,-87.668047,member,9.315861,2495.0,Tuesday,March,Weekday,Spring,One-way trip,41.583333


18. Vérifier les caractéristiques des dataframes avant l'export du dataset nettoyé

In [ ]:
df4.shape

(4528312, 20)

## Extraire le dataframe pour l'analyse

19. Exporter le dataframe pour l'utiliser par la suite dans ton Notebook d'analyse

In [ ]:
#Extract csv file (SI BESOIN)
#from google.colab import files
df4.to_csv('df_cyclistic_clean.csv', index=False)
#files.download('df_cyclistic_clean.csv')



---


# Business Case - CDA - Cyclistic (4h)



---


## Contexte

Cyclistic est une entreprise de partage de vélos à Chicago, qui a depuis élargi son offre pour inclure une flotte de 5 824 vélos géolocalisés stationnés à 692 emplacements à travers Chicago.

Les vélos peuvent être déverrouillés à une station et retournés à n'importe quelle autre station du réseau à tout moment.

Les individus achetant des passes pour une seule course ou pour une journée complète sont considérés comme des cyclistes occasionnels, tandis que ceux qui acquièrent des abonnements annuels sont reconnus comme des membres de Cyclistic.



---


## Votre mission

Tu es un analyste de données junior travaillant dans l'équipe des analystes marketing chez Cyclistic.

Le directeur du marketing pense que le succès futur de l'entreprise dépend de l'augmentation du nombre d'abonnements annuels.

Par conséquent, ton équipe souhaite comprendre comment les cyclistes occasionnels et les membres annuels utilisent différemment les vélos Cyclistic.

À partir de ces informations, ton équipe élaborera une nouvelle stratégie marketing pour convertir les cyclistes occasionnels en membres annuels.

Mais avant cela, les dirigeants de Cyclistic doivent approuver tes recommandations, donc il est essentiel d'avoir des analyses de données convaincantes et des visualisations de données professionnelles.

l y a trois questions clés qui doivent être répondues à la fin de l'étude de cas :

* Comment les membres annuels et les cyclistes occasionnels utilisent-ils les vélos Cyclistic différemment ?
* Pourquoi les cyclistes occasionnels achèteraient-ils des abonnements annuels Cyclistic ?
* Comment Cyclistic peut-il utiliser les médias numériques pour influencer les cyclistes occasionnels à devenir membres ?
* Où recommenderas-tu à Cyclistic d'implémenter des nouvelles stations pour rendre le service plus disponible ?



---
##Phase 1 : Collecte, pré-traitement et traitement des données

Dans cette phase, tu dois comprendre le processus de génération et de collecte des données, ainsi que reconnaître les différents types, formats et structures de données pour une utilisation efficace dans l'analyse des données. De plus, tu dois veiller à respecter des pratiques éthiques en matière d'analyse de données et traiter les questions de biais et de crédibilité des données.

Vous allez dans un premier temps explorer le jeu de données, dans le but d'explorer plusieurs possibilités de choix. Une fois que votre choix est fait, lancez-vous dans l'élaboration de votre tableau de bord, dans le but de convaincre un maximum les investisseurs.

**Les données sont disponibles publiquement sur un serveur AWS (https://divvy-tripdata.s3.amazonaws.com/index.html)**

Tu as été chargés de travailler avec une année entière de données, **l'année 2021**, donc il faudra récupérer les fichiers (au format CSV) contenant des données de **Janvier 2021 à Décembre 2021, un fichier pour chaque mois**.

**Pensez à bien explorer les différents fichiers et de pouvoir réfélchir à travailler sur toute l'année. Il faudra aussi penser à éliminer tous les trajets qui causent des problèmes techniques en phase de pré-traitement. N'hésitez pas à ajouter des nouvelles colonnes afin de pouvoir mener à bien vos analyses et visualisations**.




---

##Phase 2 : Analyses et visualisation des données

Pour mettre toutes les chances de votre côté et séduire les investisseurs, vous allez devoir faire les bons choix. Voici quelques questions pour vous guider:
- Quel est le pourcentage des types d'utilisateurs par rapport au total des utilisateurs ? ✅
- Y a-t-il un type de vélo préféré par les différents types d'utilisateurs ? ✅
- Quel type de vélo a la durée de trajet la plus longue entre les utilisateurs ?✅
- Quelle est la durée moyenne des trajets par type d'utilisateur ? ✅
- Quelle est la distance moyenne parcourue par type d'utilisateur ? ✅
- Quels jours sont les plus actifs pour les utilisateurs ? ✅
- Quels mois ou quelles saisons de l'année les utilisateurs ont tendance à utiliser le service de partage de vélos ?✅
- Quel type d'utilisateur a le plus grand nombre de trajets aller-retour et de trajets aller simple effectués ? ✅


---


## Phase 3 : Cartographie et recommendation d'implémentation physique

En plus de fournir une analyse complète qui vous permettra d'appuyer vos choix des recommendations à faire à l'entreprise, **prenez soin d'indiquer précisément où est meilleur de s'implanter.**

N'oubliez pas que les investisseurs, qui veulent bien placer leur argent, ne manqueront pas de soulever des points précis, pour s'assurer que votre activité soit rentable à coups sûr.  

**Dans le dataset, vous trouverez les coordonnées des stations de départ et d'arrivées.**

Comme dis plus haut, soyez créatif !   

Ne manquez pas d'étoffer vos arguments, et d'être étincelant le jour J ;)



---


## Livrables attendus

Les investisseurs souhaitent une présentation avec des slides (sous format pdf) et un tableau de bord intéractif (Power BI, Tableau ou autre outil de BI).

La présentation contiendra à minima ces éléments :
- Rappel du contexte et de la problématique
- Arguments
- Analyse exploratoire des données
- Méthodologie, outils et langages utilisés
- Votre tableau de bord contient des graphiques
- Votre tableau de bord contient au moins une visualisation de données interactive
- Votre tableau de bord contient au moins une carte représentant des informations géographiques
- Votre tableau de bord contient au moins un tableau croisé
- L'ensemble des graphiques et visuels doit être lisible par tous (prise en compte des personnes en situation de handicap visuel)
- Réponse aux questions métiers en conclusion :
  1. **Quelles sont les recommandations pour Cyclistic afin d'augmenter leur taux de clientèle, en se concentrant particulièrement sur le ciblage de leurs offres proposées et en exploitant les habitudes d'utilisation des clients ainsi que les préférences pour les vélos?**
  2. **Quel est le meilleur emplacement pour installer de nouvelles stations en se basant sur les habitudes d'utilisation des clients, les saisons et les stations les plus fréquentées ?**
